In [1]:
import pandas as pd

# Read the Excel file without headers to get raw data
excel_file = "../../data/Plant Collection Inventory and Move Management.xlsx"
df_raw = pd.read_excel(excel_file, header=None)

# Find the row with "Common Name" in the first column (header row)
header_row_idx = None
for idx in range(len(df_raw)):
    first_col = str(df_raw.iloc[idx, 0]).strip() if pd.notna(df_raw.iloc[idx, 0]) else ""
    if first_col == "Common Name":
        header_row_idx = idx
        break

# Get column names from header row and sub-header row
col_names_main = df_raw.iloc[header_row_idx].values
col_names_sub = df_raw.iloc[header_row_idx + 1].values

# Build column names by combining main and sub headers
final_columns = []
for i in range(len(col_names_main)):
    col_main = str(col_names_main[i]).strip() if pd.notna(col_names_main[i]) else ""
    col_sub = str(col_names_sub[i]).strip() if pd.notna(col_names_sub[i]) else ""
    
    if i in [0, 1, 2, 7]:  # Common Name, Scientific Name, Qty, Notes
        final_columns.append(col_main)
    elif i in [3, 4]:  # Buy New columns
        final_columns.append(f"Buy New - {col_sub}" if col_sub else col_main)
    elif i in [5, 6]:  # Move It columns
        final_columns.append(f"Move It - {col_sub}" if col_sub else col_main)
    elif i == 8:  # Display
        final_columns.append("Display")
    elif i == 9:  # Stop
        final_columns.append("Stop")
    else:
        final_columns.append(f"{col_main} - {col_sub}" if (col_main and col_sub) else (col_main or col_sub or f"Unnamed_{i}"))

print("Column names:", final_columns)

Column names: ['Common Name', 'Scientific Name', 'Qty', "Buy New - Won't Survive/Not Worth Moving", 'Buy New - Readily Available', 'Move It - Can be done by Domes staff', 'Move It - Requires consult - might not survive move', 'Notes', 'Display', 'Stop']


In [2]:
# Find dome headers and split data into separate DataFrames
# Look for rows that contain "Dome" in the first column
# Skip rows up to and including the header rows
dome_rows = []
for idx, row in df_raw.iterrows():
    first_col = str(row.iloc[0]).strip() if pd.notna(row.iloc[0]) else ""
    if "Dome" in first_col and idx > header_row_idx + 1:  # Skip header rows
        dome_rows.append((idx, first_col))

print("Found dome sections:")
for idx, dome_name in dome_rows:
    print(f"  Row {idx}: {dome_name}")

# Create a dictionary to store DataFrames for each dome
dome_dataframes = {}

# Process each dome section
for i, (dome_start_idx, dome_name) in enumerate(dome_rows):
    # Determine the end of this section (start of next dome, or end of file)
    if i + 1 < len(dome_rows):
        dome_end_idx = dome_rows[i + 1][0]
    else:
        dome_end_idx = len(df_raw)
    
    # Extract the data for this dome (skip the dome header row itself)
    dome_data = df_raw.iloc[dome_start_idx + 1:dome_end_idx].copy()
    
    # Set proper column names
    dome_data.columns = final_columns
    
    # Remove rows that are all NaN or empty
    dome_data = dome_data.dropna(how='all')
    
    # Remove rows where Common Name is NaN (these are likely empty separator rows)
    dome_data = dome_data[dome_data['Common Name'].notna()]
    
    # Reset index and set to match Excel row numbers (1-indexed)
    # The dome_start_idx is the Excel row number (0-indexed), so we add 1 for the dome header row
    # and then add 1 more for each data row to match Excel's 1-indexed system
    dome_data = dome_data.reset_index(drop=True)
    # Start index from dome_start_idx + 2 (Excel row number: dome header + 1)
    dome_data.index = dome_data.index + dome_start_idx + 2
    
    # Convert 'x' to 1 (True) and NaN to 0 (False) for boolean columns
    boolean_columns = [
        "Buy New - Won't Survive/Not Worth Moving",
        "Buy New - Readily Available",
        "Move It - Can be done by Domes staff",
        "Move It - Requires consult - might not survive move",
        "Display"
    ]
    
    for col in boolean_columns:
        if col in dome_data.columns:
            # Convert 'x' to 1, everything else (NaN, empty) to 0
            dome_data[col] = dome_data[col].apply(
                lambda x: 1 if (pd.notna(x) and str(x).strip().lower() == 'x') else 0
            )
    
    # Convert Notes NaN to empty string
    if 'Notes' in dome_data.columns:
        dome_data['Notes'] = dome_data['Notes'].fillna('').astype(str)
        # Replace 'nan' string with empty string
        dome_data['Notes'] = dome_data['Notes'].replace('nan', '')
    
    # Convert Stop NaN to 'N/A'
    if 'Stop' in dome_data.columns:
        dome_data['Stop'] = dome_data['Stop'].fillna('N/A')
        # Also replace 'nan' string with 'N/A' if it exists
        dome_data['Stop'] = dome_data['Stop'].replace('nan', 'N/A')
    
    # Add a Dome column
    dome_data['Dome'] = dome_name
    
    # Store in dictionary
    dome_dataframes[dome_name] = dome_data
    
    print(f"\n{dome_name} DataFrame shape: {dome_data.shape}")
    print(f"First few rows of {dome_name}:")
    print(dome_data.head())


Found dome sections:
  Row 5: Tropical Dome
  Row 469: Desert Dome

Tropical Dome DataFrame shape: (463, 11)
First few rows of Tropical Dome:
           Common Name         Scientific Name Qty  \
7       Chenille Plant        Acalypha hispida   1   
8      Pineapple Guava          Acca selloiana   2   
9         Magic Flower  Achimenantha 'Inferno'   2   
10  Silver-Dollar Fern     Adiantum peruvianum   3   
11     Maidenhair Fern      Adiantum raddianum   2   

    Buy New - Won't Survive/Not Worth Moving  Buy New - Readily Available  \
7                                          0                            1   
8                                          1                            1   
9                                          1                            1   
10                                         0                            1   
11                                         0                            1   

    Move It - Can be done by Domes staff  \
7                         

In [3]:
# Remove Dome column from individual dome DataFrames and create "All" combined DataFrame
dome_dataframes_with_dome = []

# Process each dome DataFrame
for dome_name in list(dome_dataframes.keys()):
    if dome_name != 'All':  # Skip if 'All' already exists
        df = dome_dataframes[dome_name]
        
        # Create a copy with Dome column for the combined DataFrame
        df_with_dome = df.copy()
        df_with_dome['Dome'] = dome_name
        dome_dataframes_with_dome.append(df_with_dome)
        
        # Remove Dome column from individual DataFrame if it exists
        if 'Dome' in df.columns:
            dome_dataframes[dome_name] = df.drop(columns=['Dome'])

# Create combined "All" DataFrame with Dome column
dome_dataframes['All'] = pd.concat(dome_dataframes_with_dome, ignore_index=True)
print(f"Created 'All' DataFrame with {dome_dataframes['All'].shape[0]} total plants")

Created 'All' DataFrame with 671 total plants


In [4]:
dome_dataframes['Tropical Dome'].head(10)

,Common Name,Scientific Name,Qty,Buy New - Won't Survive/Not Worth Moving,Buy New - Readily Available,Move It - Can be done by Domes staff,Move It - Requires consult - might not survive move,Notes,Display,Stop
7,Chenille Plant,Acalypha hispida,1,0,1,0,0,,1,N/A
8,Pineapple Guava,Acca selloiana,2,1,1,0,0,,0,3
9,Magic Flower,Achimenantha 'Inferno',2,1,1,0,0,,0,N/A
10,Silver-Dollar Fern,Adiantum peruvianum,3,0,1,1,0,,0,6
11,Maidenhair Fern,Adiantum raddianum,2,0,1,1,0,,0,5
12,Rosy Maidenhair Fern,Adiantum ternerum 'Scutum Roseum',3,0,1,1,0,,1,4
13,Diamond Maidenhair Fern,Adiantum trapeziforme,6,0,1,1,0,,0,5
14,Christmas Palm,Adonidia merrillii,2,1,1,0,0,too large to move,0,7
15,Living Vase Bromeliad,Aechmea 'Better than Bert',2,0,1,1,0,"one is potted, easily transplanted",1,3
16,Living Vase Bromeliad,Aechmea 'Black Panther',1,0,1,1,0,"potted, easily transplanted",0,1


In [5]:
dome_dataframes['Desert Dome'].head(10)

,Common Name,Scientific Name,Qty,Buy New - Won't Survive/Not Worth Moving,Buy New - Readily Available,Move It - Can be done by Domes staff,Move It - Requires consult - might not survive move,Notes,Display,Stop
471,Dwarf Hedgehog Agave,Agave stricta 'Nana',1,0,0,1,0,Doesn't seem readily available to purchase,0,N/A
472,Jelly Bean Plant,Sedum pachyphyllum,3,0,1,1,0,Readily available and easy to propagate,0,N/A
473,Spanish Bayonet (?),Yucca aloifolia (?),1,1,0,0,0,,0,N/A
474,Bushman Poison,Adenium boehmianum,1,0,0,1,0,Nice sized specimen,0,N/A
475,Zimbabwe Aloe,Aloe excelsa 'Veld',1,0,0,0,1,Large tree Aloe,0,N/A
476,Rubble Aloe,Aloe perfoliata,1,0,1,0,0,,0,N/A
477,Aloe 'Hummel Collection',Aloe sp.,1,0,0,1,0,Ideally should be identified to remain in coll...,0,N/A
478,Hercules Aloe,Aloidendron barberae x dichotoma,1,0,0,0,1,Nice large specimen,0,N/A
479,Rosary Vine,Ceropegia woodii,1,0,1,0,0,,0,N/A
480,Bear Paw,Cotyledon tomentosa 'Shake',1,0,1,0,0,,0,N/A


In [6]:
dome_dataframes['All'].head(10)

,Common Name,Scientific Name,Qty,Buy New - Won't Survive/Not Worth Moving,Buy New - Readily Available,Move It - Can be done by Domes staff,Move It - Requires consult - might not survive move,Notes,Display,Stop,Dome
0,Chenille Plant,Acalypha hispida,1,0,1,0,0,,1,N/A,Tropical Dome
1,Pineapple Guava,Acca selloiana,2,1,1,0,0,,0,3,Tropical Dome
2,Magic Flower,Achimenantha 'Inferno',2,1,1,0,0,,0,N/A,Tropical Dome
3,Silver-Dollar Fern,Adiantum peruvianum,3,0,1,1,0,,0,6,Tropical Dome
4,Maidenhair Fern,Adiantum raddianum,2,0,1,1,0,,0,5,Tropical Dome
5,Rosy Maidenhair Fern,Adiantum ternerum 'Scutum Roseum',3,0,1,1,0,,1,4,Tropical Dome
6,Diamond Maidenhair Fern,Adiantum trapeziforme,6,0,1,1,0,,0,5,Tropical Dome
7,Christmas Palm,Adonidia merrillii,2,1,1,0,0,too large to move,0,7,Tropical Dome
8,Living Vase Bromeliad,Aechmea 'Better than Bert',2,0,1,1,0,"one is potted, easily transplanted",1,3,Tropical Dome
9,Living Vase Bromeliad,Aechmea 'Black Panther',1,0,1,1,0,"potted, easily transplanted",0,1,Tropical Dome
